### Upload the Data

In [2]:
import numpy as np 
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

### Make a LinearSVC without scale

In [3]:
from sklearn.svm import LinearSVC

lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train, y_train)

C:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(random_state=42)

In [4]:
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.8348666666666666

### Scale it for better performance

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

In [9]:
lin_clf.fit(X_train_scaled, y_train)

y_pred_scaled = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred_scaled)

C:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9217333333333333

### Make a SVC

In [10]:
from sklearn.svm import SVC

svm_clf = SVC(gamma = "scale")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])

SVC()

### Randomized Search

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=8.889188010944777, gamma=0.029400829976050615 .................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .. C=8.889188010944777, gamma=0.029400829976050615, total=   0.9s
[CV] C=8.889188010944777, gamma=0.029400829976050615 .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV] .. C=8.889188010944777, gamma=0.029400829976050615, total=   0.8s
[CV] C=8.889188010944777, gamma=0.029400829976050615 .................
[CV] .. C=8.889188010944777, gamma=0.029400829976050615, total=   0.9s
[CV] C=7.349277883036248, gamma=0.024441859021298133 .................
[CV] .. C=7.349277883036248, gamma=0.024441859021298133, total=   0.8s
[CV] C=7.349277883036248, gamma=0.024441859021298133 .................
[CV] .. C=7.349277883036248, gamma=0.024441859021298133, total=   0.8s
[CV] C=7.349277883036248, gamma=0.024441859021298133 .................
[CV] .. C=7.349277883036248, gamma=0.024441859021298133, total=   0.8s
[CV] C=4.069640947464349, gamma=0.011062981065015045 .................
[CV] .. C=4.069640947464349, gamma=0.011062981065015045, total=   0.8s
[CV] C=4.069640947464349, gamma=0.011062981065015045 .................
[CV] .. C=4.069640947464349, gamma=0.011062981065015045, total=   0.8s
[CV] C=4.069640947464349, gamma=0.011062981065015045 .................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   24.7s finished


RandomizedSearchCV(cv=3, estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000029689692F88>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000029689692648>},
                   verbose=2)

In [12]:
rnd_search_cv.best_estimator_

SVC(C=5.15030053297823, gamma=0.002034844594219033)

In [ ]:
rnd_search_cv.best_score_

In [ ]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

In [ ]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

In [ ]:

y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)